### Encryption & Decryption Using Diffie Hellman Algorithm

Purpose of work:

Implement the Diffie-Hellman session key co-generation protocol. Requirements for work:
 * Development of two independent modules participating in the protocol.
 * Implement a man-in-the-middle attack
 * The report should contain a description of the protocol with an indication of the implementation features, examples of the program.

### Progress:

Diffie – Hellman Protocol (DH) ** is a cryptographic protocol that allows two or more parties to obtain a shared secret key using a communication channel unprotected from eavesdropping. The resulting key is used to encrypt further exchange using symmetric encryption algorithms

#### Implementation in language Python

In [1]:
class DiffieHellman:
    def __init__(self, g, p, private_key):
        self.g = g
        self.p = p
        self.private_key = private_key
        self.full_key = None

    def generate_public_key(self):
        public_key = (self.g ** self.private_key) % self.p
        return public_key

    def generate_full_key(self, public_key):
        full_key = (public_key ** self.private_key) % self.p
        self.full_key = full_key
        return full_key

    def encrypt_message(self, message):
        encrypted_message = ""
        for c in message:
            encrypted_message += chr(ord(c) + self.full_key)
        return encrypted_message

    def decrypt_message(self, encrypted_message):
        decrypted_message = ""
        for c in encrypted_message:
            decrypted_message += chr(ord(c) - self.full_key)
        return decrypted_message
    
    def __str__(self):
        return f'g = {self.g}\np = {self.p}\nprivate_key = {self.private_key}\nfull_key = {self.full_key}'

The DiffieHellman class provides the following methods:

`__init __ (self, g, p, private_key)` - a constructor to create a subscriber; 

`generate_public_key (self)` - method for generating a public key; 

`generate_full_key (self, public_key)` - a method for generating a full public key that takes a public key as a parameter; 

`encrypt_message (self, message)` - a message encryption method based on a simple replacement (shift) cipher;

`decrypt_message (self, encrypted_message)` - a method for decrypting a message based on a simple replacement (shift) cipher; 

`__str __ (self)` - method for displaying complete information about the subscriber

Let's set the initial parameters

In [48]:
message = "HEY DIFFIE is COOL"
g = 2
p = 211
a = 23
b = 31

In [49]:
alice = DiffieHellman(g, p, a)
bob = DiffieHellman(g, p, b)

Computing public keys

$$ga = g^a mod p$$

In [50]:
ga = alice.generate_public_key()

In [51]:
ga

92

$$gb = g^b mod p$$

In [52]:
gb = bob.generate_public_key()

In [53]:
gb

131

Public key exchange and shared secret key generation

$$K_a = gb^a mod p = g^{ab} mod p$$

In [54]:
alice_full_key = alice.generate_full_key(gb)

In [55]:
alice_full_key

133

$$K_b = ga^b mod p = g^{ab} mod p$$

In [56]:
bob_full_key = bob.generate_full_key(ga)

In [57]:
bob_full_key

133

Thus, you can see that the keys of Alice and Bob are the same.
$$K = g^{ab} mod p$$

Exchange of messages using a shared secret. For demonstration, a simple replacement cipher is used.

In [58]:
encrypted_message = alice.encrypt_message(message)

In [59]:
encrypted_message

'ÍÊÞ¥ÉÎËËÎÊ¥îø¥ÈÔÔÑ'

In [60]:
decrypted_message = bob.decrypt_message(encrypted_message)

In [61]:
decrypted_message

'HEY DIFFIE is COOL'

### Man-in-the-middle attack

A Man in the Middle (MITM) attack is a type of cryptography and computer security attack in which an attacker secretly relays and, if necessary, alters communications between two parties who believe they are directly communicating with each other. It is a method of compromising a communication channel, in which an attacker, having connected to a channel between counterparties, interferes with the transmission protocol, deleting or distorting information. 

Neither Alice nor Bob can reliably determine who their interlocutor is, so it is quite possible to imagine a case in which Bob and Alice established a connection with Eve, who impersonates Alice as Bob, and Bob introduces herself as Alice. 

Eve has her own private keys to communicate with Bob (eve_a) and Alice (eve_b)

In [32]:
message2 = "Eve HACKED IT"
eve_a = 13
eve_b = 27

Eve generates public keys

In [33]:
eve_ga = (g ** eve_a) % p
eve_gb = (g ** eve_b) % p

It passes them on to Alice and Bob, who generate shared secret keys. But in this case, they will not be the same, since Eve has one shared key with Alice (who believes that it is Bob) and one shared key with Bob (who believes that it is Alice). And, therefore, she can receive any message from Alice for Bob, decrypt it with the key, read, change, encrypt with the key, and send it to Bob. Thus, forgery can go unnoticed for a very long time.

In [34]:
eve_alice_full_key = alice.generate_full_key(eve_gb)

In [35]:
eve_alice_full_key

68

In [36]:
eve_bob_full_key = bob.generate_full_key(eve_ga)

In [37]:
eve_bob_full_key

175

In [38]:
print(alice)

g = 2
p = 211
private_key = 23
full_key = 68


In [39]:
print(bob)

g = 2
p = 211
private_key = 31
full_key = 175


Alice encrypts the message with her full secret key

In [40]:
encrypted_message2 = alice.encrypt_message(message2)

In [41]:
encrypted_message2

'\x89º©d\x8c\x85\x87\x8f\x89\x88d\x8d\x98'

Eve decrypts Alice's message and reads it

In [42]:
eve_decrypted_message = ""
for c in encrypted_message2:
    eve_decrypted_message += chr(ord(c) - eve_alice_full_key)

In [43]:
eve_decrypted_message

'Eve HACKED IT'

Eve modifies Alice's message and encrypts it with Bob's shared key

In [44]:
eve_message = "HEY I DID MITM"
eve_encrypted_message = ""
for c in eve_message:
    eve_encrypted_message += chr(ord(c) + eve_bob_full_key)

In [45]:
eve_encrypted_message

'÷ôĈÏøÏóøóÏüøăü'

Bob decrypts the message and receives not at all what Alice sent him, but the message sent by Eve.

In [46]:
decrypted_message2 = bob.decrypt_message(eve_encrypted_message)

In [47]:
decrypted_message2

'HEY I DID MITM'

### Conclusions

Implemented the Diffie-Hellman session key joint generation protocol in Python. A man-in-the-middle attack was also implemented, as a result of which it was found that the Diffie-Hellman protocol is vulnerable to this attack. Therefore, to use it, it is necessary to use additional security methods (digital signature, hash function, etc.).